In [1]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
using Printf
using JSON
using CSV
using DataFrames
using LaTeXStrings
using BenchmarkTools
LaTeXStrings

LaTeXStrings

In [2]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
include("../src/SailorMoon.jl")   # relative path to main file of module
param3b = SailorMoon.dynamics_parameters()
paramMulti = SailorMoon.multi_shoot_parameters(param3b)
plotly()
fs = 16

┌ Warning: Replacing docs for `Main.SailorMoon.integrate_rk4 :: Union{Tuple{ODEProblem, Real}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool}, Tuple{ODEProblem, Real, Union{Nothing, Vector{T} where T}, Bool, Bool}}` in module `Main.SailorMoon`
└ @ Base.Docs docs\Docs.jl:240
┌ Warning: For saving to png with the `Plotly` backend `PlotlyBase` and `PlotlyKaleido` need to be installed.
│   err =
│    ArgumentError: Package PlotlyBase not found in current path:
│    - Run `import Pkg; Pkg.add("PlotlyBase")` to install the PlotlyBase package.
│    
└ @ Plots C:\Users\yujit\.julia\packages\Plots\nuwp4\src\backends.jl:545


16

## obtain an initial guess

In [3]:
### PARAMETERS ###################################
# csv file to load the initial solution
filename = "../run/data/grid_search_Tsit5_0525_EMrotThrust.csv"
# dv_dir function corresponding to the csv file 
dir_func = SailorMoon.dv_EMrotdir_sb1frame 

# parameters
param3b = SailorMoon.dynamics_parameters()
paramMulti = SailorMoon.multi_shoot_parameters(param3b)

n_arc = 5
##################################################

if dir_func == SailorMoon.dv_no_thrust
    τ_ig = 0.0
else 
    τ_ig = 1.0
end

# load initial guess
df = DataFrame(CSV.File(filename))

# maybe want to use "for row in eachrow(df)" to automate the process...? 
row = df[21,:]


Row,id,phi0,epsr,epsv,thetasf,rp_kep,ra_kep,alpha,ra,dt1,dt2,x_lpo,y_lpo,z_lpo,xdot_lpo,ydot_lpo,zdot_lpo,m_lpo,x_ra,y_ra,z_ra,xdot_ra,ydot_ra,zdot_ra,m_ra,x_rp,y_rp,z_rp,xdot_rp,ydot_rp,zdot_rp,m_rp,x_lr,y_lr,z_lr,xdot_lr,ydot_lr,zdot_lr,m_lr,t_lr,tof,lfb
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
21,21.0,0.397935,1.0e-5,1.0e-5,5.44543,0.00808627,2.2964,-2.60377,2.84033,9.48719,6.08247,388.073,-0.902908,-1.62381e-21,0.717073,-0.644585,-1.49526e-21,1.0,391.481,0.995379,-4.82231e-21,-0.0136708,0.0414753,-5.25646e-22,1.01762,388.821,-0.0143727,0.0,7.99321,-13.4007,0.0,1.03253,388.821,-0.0143727,1.15655e-23,7.99321,-13.4007,2.66139e-19,1.03253,15.5697,15.5697,0.0


In [4]:
x0, lx, ux = SailorMoon.make_ig_bounds2(row, τ_ig, paramMulti.n_arc)
fitness!, ng, lg, ug, eval_sft = SailorMoon.get_fitness2(dir_func, paramMulti, x0);

In [5]:
@time DF = SailorMoon.get_jacobian(eval_sft, x0, ng)

 35.059732 seconds (281.43 M allocations: 9.457 GiB, 6.46% gc time, 0.09% compilation time)


16×97 Matrix{Float64}:
 43983.0          26367.3            0.0         …   0.0           0.0
  6015.69          3274.65           0.0             0.0           0.0
     7.34483e-17      5.51386e-17  283.853           0.0           0.0
 23660.5          14131.5            0.0             0.0           0.0
  2198.21          1278.82           0.0             0.0           0.0
     6.31382e-18      4.81859e-18   26.5385      …   0.0           0.0
    -2.10942e-9      -1.77636e-9     0.0             0.0           0.0
     0.0              0.0            0.0            -0.00462438    0.0
     0.0              0.0            0.0            -0.00526156    0.0
     0.0              0.0            0.0             8.85845e-26   0.00511223
     0.0              0.0            0.0         …   0.0191705     0.0
     0.0              0.0            0.0             0.0214523     0.0
     0.0              0.0            0.0            -7.04899e-25  -0.0204717
     0.0              0.0            0.0 

In [6]:
@time inv(DF*transpose(DF))

  0.443179 seconds (708.25 k allocations: 41.003 MiB, 99.80% compilation time)


16×16 Matrix{Float64}:
  0.186576     -0.0482387     -3.49713e-22  …   14.4254        -0.00133494
 -0.0482387     0.0310564      4.94123e-21       3.07823       -0.000287695
 -3.49714e-22   4.94123e-21    3.38827           4.62845e-18   -4.28449e-22
 -0.425781      0.121679       2.45641e-21      -0.72536        6.69084e-5
  0.987432     -0.42829       -4.02741e-20      77.5666        -0.00719207
  3.75286e-21  -5.28447e-20  -36.2405       …   -4.86342e-17    4.50129e-21
 -0.0525058     0.017368       6.51121e-21       3.88391       -0.000362449
 -0.0220838    -0.00599041    -4.15122e-20      -4.6781         0.000434903
  0.272214     -0.0787294     -3.45118e-20      10.1714        -0.000940905
  1.30701e-21  -1.35325e-20   -9.49378          -1.27811e-17    1.18297e-21
 -0.0985198     0.0302872      8.00787e-20  …   20.9673        -0.00195285
  0.205149     -0.137883       3.77171e-20     -17.0552         0.00159435
 -4.59059e-21  -1.08753e-20   -3.80707          -5.39632e-18    4.9994

In [7]:
@time x_iter = SailorMoon.newton_update(x0, eval_sft(x0), DF)

  0.105837 seconds (1.33 M allocations: 42.015 MiB, 26.03% compilation time)


97-element Vector{Float64}:
 388.81062948076504
  -0.020717445993906777
   2.0947886464430783e-23
   3.141710955979362
  -5.265178330103744
   2.643216553706365e-19
   1.1029308062294174
   1.8140235566302022e-5
   3.110168818813142
   1.0
   0.0
   0.0
   1.0
   ⋮
   1.0063205738305894
  -0.00991371828898728
  -4.6315137906087596e-21
   1.0088282528769152
  -0.005732211642198792
  -2.8040449284029977e-21
   1.009196463255867
  -0.002446480898688611
  -1.0397450136324417e-21
   1.0083029427682517
   2.949379356849298e-5
   4.362299701764637e-22

In [ ]:
xs, fs, converged = SailorMoon.differential_correction(x0, eval_sft)

iter #